In [1]:
import numpy as np
import pathlib
from PIL import Image

In [2]:
textures_dir = "textures"
directories = ["bricks", "leather", "mosaic", "wood"]
current_dir = pathlib.Path().resolve()

In [3]:
def create_sample_dir_for(texture_type, filename):
    p = current_dir / 'samples' / texture_type / filename
    p.parent.mkdir(parents=True, exist_ok=True)
    return p


def read_image():
    for dir in directories:
        p = current_dir / textures_dir / dir
        for f in p.iterdir():
            yield dir, Image.open(f)

In [4]:
def split_image_into_samples(img: Image, width: int, height: int):
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    img_arr = np.array(img)
    iw, ih, _ = img_arr.shape
    samples = []

    splitted_ax_0 = np.split(img_arr, np.arange(width, iw, width))
    if iw % width != 0:
        splitted_ax_0 = splitted_ax_0[:-1]

    hindices = np.arange(height, ih, height)
    for a in splitted_ax_0:
        splitted_ax_1 = np.split(a, hindices, axis=1)
        if ih % height != 0:
            splitted_ax_1 = splitted_ax_1[:-1]
        samples.extend(map(Image.fromarray, splitted_ax_1))
    return samples

In [5]:
for texture_type, img in read_image():
    sample_size = (128, 128)
    sampled = split_image_into_samples(img, *sample_size)
    for i, sample in enumerate(sampled):
        sample.save(create_sample_dir_for(texture_type, f'sample{i}.jpg'))